In [1]:
#importing all the required packages for analysis.
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import logging

In [10]:
#importing the data to analyze
import requests
def read_web_csv(URL):
      """
  Attempts to read a CSV file from a provided URL and returns a pandas DataFrame.

  Args:
      URL (str): The URL of the CSV file.

  Returns:
      pandas.DataFrame: The DataFrame containing the data from the CSV file if successful, 
                         otherwise raises an exception.

  Raises:
      requests.exceptions.RequestException: If there's an error downloading the file.
      pd.errors.EmptyDataError: If the URL does not point to a valid CSV file.
      Exception: Any other unexpected exception during processing.
  """
      try:

            #Reading the downloaded file content and converting it into a dataframe
            df = pd.read_csv(URL)
            print("CSV file read successfully!")
            return df
      except requests.exceptions.RequestException as e:
            print("Failed to load CSV file: {e}")
            raise e 
      except pd.errors.EmptyDataError as e:
            print("The CSV file is not valid. Please check the URL and try again.")
            raise e
      except Exception as e:
            print("Failed to read the CSV file. Error: {e}")
            raise e



In [11]:
md_df = read_web_csv('https://raw.githubusercontent.com/Professor-blu/Data_science-_python_assignments/main/python_practical_exam/MD_agric_exam-4313.csv')

CSV file read successfully!


In [12]:
md_df.head()

,Field_ID,Elevation,Rainfall,Min_temperature_C,Max_temperature_C,pH,Pollution_level,Plot_size,Annual_yield,Crop_type
0,1162,494.95615,1507.6,-5.4,31.0,6.859436,0.007034,3.6,1.617421,coffee
1,5108,663.73390,581.0,-4.7,30.9,5.603219,0.289643,4.2,2.532497,potato
2,3504,396.87990,1715.1,-6.1,31.7,5.774116,0.000409,2.6,1.262207,banana
3,5351,594.80370,1748.0,-4.3,33.6,6.477415,0.088777,7.9,4.351564,wheat
4,905,609.49800,1395.8,-4.5,31.3,5.419586,0.050023,10.8,5.034791,cassava


In [13]:
def count_unique_crop_type(df):
    """
    Counts the number of unique crop types in the specified dataframe and returns the result.
    
    Args:
        df (pandas.DataFrame): The input dataFrame

    Returns:
        int: The number of unique crop types in the specified dataframe.

    Raises:
        ValueError: If the input dataframe is empty.

    """
    #Check if column "Crop_type" exists
    if "Crop_type" not in df.columns:
        raise ValueError("The specified dataframe does not contain the 'Crop_type' column.")
    #Counts the number of unique crop types
    return df["Crop_type"].nunique()

In [14]:
count_unique_crop_type(md_df)

8

In [22]:
def max_annual_yield(df, crop_type):
    """
    Returns the maximum annual yield for the specified crop type in the dataframe.

    Args:
        df(pandas.DataFrame): The input dataframe.
        crop_type(str): The crop type for which to calculate the maximum annual yield.

    Returns:
        float: The maximum annual yield rounded off to 2 decimal places.

    Raises:
        ValueError: if the specified crop type is not present in the dataframe.
    """
    #Check if column "Crop_type" exists
    if "Crop_type" not in df.columns:
        raise ValueError("The specified dataframe does not contain the crop_type column.")
    #checks if the specified crop type is present in the "Crop_type" column
    if crop_type not in df["Crop_type"].unique():
        raise ValueError("The specified crop type is not present in the dataframe.")
    else:
        #returns the maximum annual yield for specified crop type rounded off to 2 decimal places
        max_yield = df.loc[df["Crop_type"] == crop_type, "Annual_yield"].max()
        return round(max_yield, 2)
    

In [23]:
max_annual_yield(md_df, "wheat")

8.99

In [43]:
def tot_rainfall(df, pollution_level):
    """
    Calculates the total rainfall for crop types with pollution level greater than the specified threshold.

    Args:
        df(pandas.DataFrame): The input dataframe.
        pollution_level(int): The pollution level threshold.

    Returns:
        float: The total rainfall.

    Raise:
        ValueError: If the "Crop_type", "Pollution_level", "Rainfall" columns are not present in the specified dataframe.
    """
    #Check if columns "Crop_type", "Pollution_level", "Rainfall" exists.
    if "Crop_type"not in df.columns or "Pollution_level" not in df.columns or "Rainfall" not in df.columns:
        raise ValueError("The specified dataframe does not contain the required columns.")
    else:
        #Groups the dataframe by "Crop_type".
        grouped_data = df.groupby("Crop_type")

        #Finds the average pollution level for each group.
        avg_pollution_level = grouped_data["Pollution_level"].mean()

          # Filter crop types with average pollution level above the threshold
        high_pollution_crops = avg_pollution_level[avg_pollution_level > pollution_level]

        # Filter the original data for these crop types
        filtered_df = df[df['Crop_type'].isin(high_pollution_crops.index)]

        # Calculate the total rainfall for each crop type in the filtered data
        total_rainfall_high_pollution = filtered_df['Rainfall'].sum()

        return total_rainfall_high_pollution
               

In [44]:
tot_rainfall(md_df, 0.2)

941959.7

In [47]:
def calculate_temp_range(df, field_id):
    """
    Calculates the temperature range for the specified field ID.

    Args:
        df(pandas.DataFrame): The input dataframe.
        field_id(int): The field Id

    Retruns:
        float: The temperature range.
    
    Raises:
        ValueError: If the Field_ID column is not present in the specified dataframe.
        ValueError: If the specified field ID is not present in the "Field_ID" column.
    """
    #Check if "Field_ID" column exists.
    if "Field_ID" not in df.columns:
        raise ValueError("The specified dataframe does not contain the 'Field_ID' column.")
    else:
       #Checks if the specified field ID is present in the "Field_ID" column.
        if field_id not in df["Field_ID"].unique():
            raise ValueError("The specified field Id is not present in the dataframe.")
        else:
            #Filters the dataframe by the specified field ID
            filtered_df = df[df["Field_ID"] == field_id]

            #Calculates the temperature range
            temp_range = filtered_df["Max_temperature_C"] - filtered_df["Min_temperature_C"]

            return temp_range

In [50]:
temp_1 =calculate_temp_range(md_df, 1458)
temp_2 = calculate_temp_range(md_df, 1895)
temp_3 = calculate_temp_range(md_df, 5443)
print(temp_1, temp_2, temp_3)

135    40.7
dtype: float64 644    35.7
dtype: float64 892    33.4
dtype: float64


In [53]:
def tot_plot_size_low_pH(df, pH):
    """
    Calculates the total plot size for fields with pH lower than the specified threshold.

    Args:
        df(pandas.DataFrame): The input dataframe.
        pH(float): The pH threshold.

    Returns:
        float: The total plot size.

    Raises:
        ValueError: If the "Field_ID", "pH", "Plot_size" columns are not present in the specified dataframe.
    """
    
    #Checks if "Field_ID", "pH", "Plot_size" columns exist.
    if "Field_ID" not in df.columns or "pH" not in df.columns or "Plot_size" not in df.columns:
        raise ValueError("The specified dataframe does not contain the required columns.")
    else:
        #Filters the dataframe by the specified pH
        filtered_df = df[df["pH"] < pH]

        #Calculates the total plot size
        total_plot_size = filtered_df["Plot_size"].sum()

        return total_plot_size

In [54]:
tot_plot_size_low_pH(md_df, 5.5)

1731.8999999999999

In [57]:
#DataFrame that includes entries with a min_temp < -5 and max_temp > 30
md_df
filtered_df = md_df[(md_df['Min_temperature_C']< -5) & (md_df['Max_temperature_C']> 30)]
number_of_rows = filtered_df.shape[0]
print(number_of_rows)

319


In [62]:
def std_dev_large_plots(df):
    """
    Calculates the standard deviation of plot sizes for fields with plot sizes greater than the median.

    Args:
        df(pandas.DataFrame): The input dataframe.

    Returns:
        float: The standard deviation of the rainfall rounded off to 2 decimal places.
    
    Raises:
        ValueError: If the "Field_ID", "Plot_size", "Rainfall" columns are not present in specified dataframe.
    """
    #Checks if "Field_ID", "Plot_size" columns exist.
    if "Field_ID" not in df.columns or "Plot_size" not in df.columns or "Rainfall" not in df.columns:
        raise ValueError("The specified dataframe does not contain the required columns.")
    else:
        #Calculates the median of the plot sizes.
        plot_size_median = df['Plot_size'].median()

        #Filters the dataframe by the plot sizes greater than the median
        filtered_df = df[df['Plot_size'] > plot_size_median]
        
        #Calculates the standard deviation of the plot sizes
        std_dev = filtered_df['Rainfall'].std()

        return round(std_dev, 2)

In [63]:
std_dev_large_plots(md_df)

470.09

In [66]:
def count_crop_type_and_temp(df):
    """
    Calculates the number of crop types and the most common maximum temperature.

    Args:
        df(pandas.DataFrame): The input dataframe.

    Returns:
        dict: A dictionary containing the crop type, the number of the crop types and the most common maximum temerature.
    
    Raises:
        ValueError: If the "Crop_type", "Max_temperature_C" columns are not present in the specified dataframe.
    """
    #Checks if the "Crop_type", "Max_temperature_C" columns exist.
    if 'Crop_type' not in df.columns or 'Max_temperature_C' not in df.columns:
        raise ValueError("The specified dataframe does not contain the required columns.")
    else:
        #Calculates the number of crop types.
        crop_count = df.groupby('Crop_type')['Crop_type'].count().to_dict()

        #Calculates the most common maximum temperature.
        max_temp = df['Max_temperature_C'].mode()[0]

        return {
            'Crop counts' : crop_count,
            'Most common maximum temperature' : max_temp
        }

In [67]:
count_crop_type_and_temp(md_df)

{'Crop counts': {'banana': 103,
  'cassava': 141,
  'coffee': 99,
  'maize': 82,
  'potato': 146,
  'rice': 48,
  'tea': 137,
  'wheat': 244},
 'Most common maximum temperature': 30.7}

In [68]:
def crop_type_sum(df, crop_type):
    """
    Calculate the integer value of a crop type based on the length of its name.

    Args:
        df(pandas.DataFrame): The input dataframe.
        crop_type(str): The name of the crop type.

    Returns:
        int: The integer value of the crop type.

    Raises:
        ValueError: If the "Crop_type" column is not present in the specified dataframe.
        ValueError: If the specified crop type is not present in the "Crop_type" column.
    """

    #Checks if the "Crop_type" column exists.
    if 'Crop_type' not in df.columns:
        raise ValueError("The specified dataframe does not contain the 'Crop_type' column.")
    
    #Checks if the crop_type is present in the "Crop_type" column.
    if crop_type not in df['Crop_type'].unique():
        raise ValueError("The specified crop type is not present in the dataframe.")
    else:
        #Calculate the integer value of the crop type.
        crop_type_int = len(crop_type)
        
        return crop_type_int

In [71]:
banana_int = crop_type_sum(md_df, 'banana')
cassava_int = crop_type_sum(md_df, 'cassava')
coffee_int = crop_type_sum(md_df, 'coffee')
maize_int = crop_type_sum(md_df, 'maize')
potato_int = crop_type_sum(md_df, 'potato')
rice_int = crop_type_sum(md_df, 'rice')
tea_int = crop_type_sum(md_df, 'tea')
wheat_int = crop_type_sum(md_df, 'wheat')
sum_int = banana_int + cassava_int + coffee_int + maize_int + potato_int + rice_int + tea_int + wheat_int
sum_int

42